In [ ]:
import os
from openai import OpenAI
import pandas as pd

In [3]:
api_key = "здесь был ключ"
os.environ["OPENAI_API_KEY"] = api_key


client = OpenAI()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Как устроена модель GPT-3?",
        }
    ],
    # model="gpt-4-1106-preview",
    model="gpt-3.5-turbo",
)

In [4]:
print(chat_completion.choices[0].message.content)

GPT-3 (Generative Pre-trained Transformer 3) - это языковая модель, разработанная компанией OpenAI. Она основана на технологии трансформера, которая представляет собой архитектуру нейронной сети, специально разработанную для работы с последовательными данных, такими как текст.

Модель GPT-3 использует методы машинного обучения, а именно, технику предварительного обучения на больших объемах текста. Сначала модель подвергается предварительному обучению на огромном корпусе текстов из интернета, чтобы научиться различным языковым структурам и связям.

После предварительного обучения модель дополняется небольшим количеством обучающих данных для выполнения конкретной задачи. Таким образом, GPT-3 может выполнять различные языковые задачи, такие как генерация текста, отвечая на вопросы, перевод текста, составление стихотворений и многое другое.

Архитектура трансформера в GPT-3 состоит из нескольких слоев, называемых трансформерными блоками. Они состоят из множества внимания (attention mechani

In [88]:
def chat_gpt(prompt, engine):

    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=engine,
    )
    return chat_completion.choices[0].message.content

In [65]:
# df = pd.read_csv("articles_new.csv")
df = pd.read_csv("sampling_arts.csv")

In [66]:
df

,link,title,article
0,https://www.d-kvadrat.ru/novosti/23002,В «Ижавиа» рассказали о правилах провоза лыжно...,В Ижавиа рассказали о правилах провоза спортив...
1,https://www.d-kvadrat.ru/novosti/23004,Сегодня индекс МосБиржи продолжит проторговыва...,Сегодня индекс МосБиржи продолжит проторговыва...
2,https://www.d-kvadrat.ru/novosti/23000,Алан Лушников: калибр армейских автоматов нужн...,Президент концерна Калашников Алан Лушников за...
3,https://www.d-kvadrat.ru/novosti/23007,Bloomberg: рецессия в России продлится до III ...,Российская экономика начнет расти только в тре...
4,https://www.d-kvadrat.ru/novosti/23008,Подведены предварительные итоги здравоохранени...,О предварительных итогах работы отрасли здраво...
...,...,...,...
4356,https://www.d-kvadrat.ru/novosti/28709,Удмуртия вошла в ТОП-5 «зеленых» регионов России,Удмуртия продемонстрировала высокий уровень ра...
4357,https://www.d-kvadrat.ru/novosti/28712,С приближением заседания ЦБ инверсия кривой ОФ...,С приближением заседания ЦБ инверсия кривой ОФ...
4358,https://www.d-kvadrat.ru/novosti/28708,100 школьников Ижевска посетили музей ИРЗ в ра...,В ноябре более школьников классов посетили...
4359,https://www.d-kvadrat.ru/novosti/28711,Сотрудник ИРЗ на доске почета УР,К Дню государственности Удмуртии была обновлен...


In [67]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd

# df = pd.read_csv('articles_balanced.csv')
stopwords = nltk.corpus.stopwords.words('russian')

# Инициализируйте TfidfVectorizer для извлечения признаков из текстов
vectorizer = TfidfVectorizer(stop_words=stopwords, max_features=100)

# Преобразуйте текстовые данные в матрицу TF-IDF признаков
X = vectorizer.fit_transform(df['article'])

# Определите количество кластеров (в данном случае, предположим, что у нас 5 тем)
num_clusters = 5

# Примените K-средних для кластеризации
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)



# Добавьте метки кластеров в DataFrame
df['кластер'] = kmeans.labels_

# Выведите примеры текстов для каждого кластера
for i in range(num_clusters):
    cluster_samples = df[df['кластер'] == i]['article'].sample(min(5, len(df[df['кластер'] == i])))
    print(f'Cluster {i + 1}:\n{cluster_samples}\n')

# Вычислите silhouette score, чтобы оценить качество кластеризации (от 0 до 1, где 1 - лучше)
silhouette_avg = silhouette_score(X, kmeans.labels_)
print(f'Silhouette Score: {silhouette_avg}')

/Users/DNK/PycharmProjects/ML+DA/venv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster 1:
1077    Помощь фронту, Своих не бросаем, КнигиДонбассу...
4012    Такие результаты опубликовала прессслужба Удму...
2824    С момента запуска антиалкогольного проекта Тре...
2895    Самки Юки и Аме родили двух оленят  мальчика и...
3473    Ижевск планирует пополнить свой общественный т...
Name: article, dtype: object

Cluster 2:
2610    Временно исполняющий полномочия Главы города И...
2882    С такой просьбой обратились жители микрорайона...
567     С  по  января  изза резкого похолодания отмени...
3840    На данный момент в Ижевске существуют  действу...
3073    В предстоящие выходные ограничат движение тран...
Name: article, dtype: object

Cluster 3:
1627    Индекс МосБиржи , продолжил уверенно расти, пр...
4184    Индекс МосБиржи завершил торги в понедельник н...
1836    Индекс МосБиржи , обновил максимумы сентября п...
4143    В четверг индекс МосБиржи опустился ниже отмет...
321     Индекс МосБиржи , во вторник завершил день в с...
Name: article, dtype: object

Cluster

In [68]:
df

,link,title,article,кластер
0,https://www.d-kvadrat.ru/novosti/23002,В «Ижавиа» рассказали о правилах провоза лыжно...,В Ижавиа рассказали о правилах провоза спортив...,3
1,https://www.d-kvadrat.ru/novosti/23004,Сегодня индекс МосБиржи продолжит проторговыва...,Сегодня индекс МосБиржи продолжит проторговыва...,2
2,https://www.d-kvadrat.ru/novosti/23000,Алан Лушников: калибр армейских автоматов нужн...,Президент концерна Калашников Алан Лушников за...,3
3,https://www.d-kvadrat.ru/novosti/23007,Bloomberg: рецессия в России продлится до III ...,Российская экономика начнет расти только в тре...,3
4,https://www.d-kvadrat.ru/novosti/23008,Подведены предварительные итоги здравоохранени...,О предварительных итогах работы отрасли здраво...,0
...,...,...,...,...
4356,https://www.d-kvadrat.ru/novosti/28709,Удмуртия вошла в ТОП-5 «зеленых» регионов России,Удмуртия продемонстрировала высокий уровень ра...,0
4357,https://www.d-kvadrat.ru/novosti/28712,С приближением заседания ЦБ инверсия кривой ОФ...,С приближением заседания ЦБ инверсия кривой ОФ...,4
4358,https://www.d-kvadrat.ru/novosti/28708,100 школьников Ижевска посетили музей ИРЗ в ра...,В ноябре более школьников классов посетили...,3
4359,https://www.d-kvadrat.ru/novosti/28711,Сотрудник ИРЗ на доске почета УР,К Дню государственности Удмуртии была обновлен...,0


In [69]:
# все у кого кластер 2 переписать на 3
df['кластер'] = df['кластер'].replace(2, 4)

In [70]:
print(df['кластер'].value_counts())

кластер
3    1551
0    1544
1     802
4     464
Name: count, dtype: int64


In [71]:
def solve_disbalance_clusters(df, num_clusters):
    df_balanced = pd.DataFrame()
    for i in range(num_clusters):
        cluster_data = df[df['кластер'] == i]
        if len(cluster_data) > 260:
            cluster_data = cluster_data.sample(260)
            df_balanced = pd.concat([df_balanced, cluster_data])
        else:
            cluster_data = cluster_data.sample(len(cluster_data))
            df_balanced = pd.concat([df_balanced, cluster_data])
    return df_balanced

In [72]:
df_balanced = solve_disbalance_clusters(df, num_clusters)

In [73]:
df_balanced.to_csv("articles_balanced.csv", index=False)
# df_balanced.to_csv("sampling_arts.csv", index=False)

In [74]:
#вывести по одному тексту из каждого кластера
for i in range(num_clusters):
    cluster_samples = df_balanced[df_balanced['кластер'] == i]['article'].sample(min(1, len(df_balanced[df_balanced['кластер'] == i])))
    print(f'Cluster {i + 1}:\n{cluster_samples}\n')

Cluster 1:
700    Ночью  января в деревне Бажаново произошло воз...
Name: article, dtype: object

Cluster 2:
3045    В рамках активной реализации национального про...
Name: article, dtype: object

Cluster 3:
Series([], Name: article, dtype: object)

Cluster 4:
563    Ровно  лет назад,  января  года, состоялся пер...
Name: article, dtype: object

Cluster 5:
832    В понедельник,  января, на рынке облигаций фед...
Name: article, dtype: object



In [ ]:
df = pd.read_csv("articles_new.csv")

In [75]:
# articles_df = pd.read_csv("articles_balanced.csv")
articles_df = pd.read_csv("sampling_arts.csv")
articles_df

,link,title,article,кластер
0,https://www.d-kvadrat.ru/novosti/24796,В Удмуртии простились с Константином Кутяновым...,марта в Удмуртии простились с Константином Ку...,0
1,https://www.d-kvadrat.ru/novosti/26425,Спасатели оперативно освободили закрытого в ма...,"Вчера, июня, спасатели Удмуртии среагировали ...",0
2,https://www.d-kvadrat.ru/novosti/28679,Дети участников СВО и из «Движения первых» пос...,На выставке ребятам особенно запомнился павиль...,0
3,https://www.d-kvadrat.ru/novosti/27700,В Удмуртии начнут отмечать День мировой юстици...,Такое решение было принято сентября в ходе се...,0
4,https://www.d-kvadrat.ru/novosti/23134,Сегодня на сессии Госсовета Удмуртии депутаты ...,"На прошедшей сегодня, ноября, сессии Государс...",0
...,...,...,...,...
1035,https://www.d-kvadrat.ru/novosti/26396,Пара доллар-рубль перешла к консолидации в уз...,Пара долларрубль обновив в рамках недели макси...,4
1036,https://www.d-kvadrat.ru/novosti/25074,Индекс МосБиржи продолжил уверенно расти,"Индекс МосБиржи , продолжил уверенно расти, пр...",4
1037,https://www.d-kvadrat.ru/novosti/27527,Потенциал для восстановления котировок ОФЗ сох...,Потенциал для восстановления котировок ОФЗ сох...,4
1038,https://www.d-kvadrat.ru/novosti/27767,Российский рынок в пятницу продолжил восходящу...,Российский рынок в пятницу продолжил восходящу...,4


In [76]:
def split_df_into_three(df):
    df_1 = df.iloc[:int(len(df)/3)]
    df_2 = df.iloc[int(len(df)/3):int(len(df)/3)*2]
    df_3 = df.iloc[int(len(df)/3)*2:]
    return df_1, df_2, df_3

In [77]:
df_1, df_2, df_3 = split_df_into_three(articles_df)

In [ ]:
# df_1.index = range(len(df_1))
# df_2.index = range(len(df_2))
# df_3.index = range(len(df_3))

In [99]:
df_1

,link,title,article,кластер,emotion
0,https://www.d-kvadrat.ru/novosti/24796,В Удмуртии простились с Константином Кутяновым...,марта в Удмуртии простились с Константином Ку...,0,
1,https://www.d-kvadrat.ru/novosti/26425,Спасатели оперативно освободили закрытого в ма...,"Вчера, июня, спасатели Удмуртии среагировали ...",0,
2,https://www.d-kvadrat.ru/novosti/28679,Дети участников СВО и из «Движения первых» пос...,На выставке ребятам особенно запомнился павиль...,0,
3,https://www.d-kvadrat.ru/novosti/27700,В Удмуртии начнут отмечать День мировой юстици...,Такое решение было принято сентября в ходе се...,0,
4,https://www.d-kvadrat.ru/novosti/23134,Сегодня на сессии Госсовета Удмуртии депутаты ...,"На прошедшей сегодня, ноября, сессии Государс...",0,
...,...,...,...,...,...
341,https://www.d-kvadrat.ru/novosti/24627,Удмуртский биатлонист Ильназ Мухамедзянов выиг...,Удмуртский биатлонист Ильназ Мухамедзянов выиг...,1,
342,https://www.d-kvadrat.ru/novosti/26148,В Ижевске начались ремонтные работы тротуаров ...,На трех тротуарах Ижевска начались ремонтные р...,1,
343,https://www.d-kvadrat.ru/novosti/26038,Белгородский губернатор: В Белгородскую област...,Владимиру Путину доложили о попытке прорыва ук...,1,
344,https://www.d-kvadrat.ru/novosti/23123,​124 тонны покрышек вывезено с контейнерных пл...,С августа по ноябрь в Ижевске был организован ...,1,


In [102]:
df_2

,link,title,article,кластер,emotion
0,https://www.d-kvadrat.ru/novosti/26886,Приемка школ и учреждений дополнительного обра...,июля был дан старт приемке школ к новому учеб...,1,
1,https://www.d-kvadrat.ru/novosti/26849,В Ижевске стартовала акция «Собери ребенка в ш...,Она направлена на помощь в подготовке к школе ...,1,
2,https://www.d-kvadrat.ru/novosti/23549,Прокуратуру Индустриального района Ижевска воз...,декабря Генпрокурор России подписал указ о на...,1,
3,https://www.d-kvadrat.ru/novosti/26161,НОВОСТНАЯ СРЕДА,Прекращение полномочий Олега Бекмеметьева и пе...,1,
4,https://www.d-kvadrat.ru/novosti/26904,Бывший глава Ижевска получил меру пресечения: ...,Индустриальный районный суд г. Ижевска принял ...,1,
...,...,...,...,...,...
341,https://www.d-kvadrat.ru/novosti/27965,Количество нарушений ПДД на переездах ГЖД сниз...,За январьсентябрь года на переездах Горьковск...,3,
342,https://www.d-kvadrat.ru/novosti/28455,Студенты из Удмуртии могут выиграть по миллион...,Студенты учреждений среднего профессионального...,3,
343,https://www.d-kvadrat.ru/novosti/25854,10 мая ЧВК «Вагнер» покинет Бахмут (Артёмовск)...,Частная военная компания Вагнер мая покинет г...,3,
344,https://www.d-kvadrat.ru/novosti/24974,Посетителей «Ижевских терм» пропустили в ТРЦ «...,"марта у единственного входа в ТРЦ Италмас, ко...",3,


In [83]:
print(df_1['emotion'].value_counts())
print("   ")
print(df_2['emotion'].value_counts())
print("   ")
print(df_3['emotion'].value_counts())



emotion
    346
Name: count, dtype: int64
   
emotion
    346
Name: count, dtype: int64
   
emotion
    348
Name: count, dtype: int64


In [82]:
df_1['emotion'] = ''
df_2['emotion'] = ''
df_3['emotion'] = ''

/var/folders/rn/rmhpxg1178d0qhdyy8l2xg200000gp/T/ipykernel_6259/491618557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['emotion'] = ''
/var/folders/rn/rmhpxg1178d0qhdyy8l2xg200000gp/T/ipykernel_6259/491618557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['emotion'] = ''
/var/folders/rn/rmhpxg1178d0qhdyy8l2xg200000gp/T/ipykernel_6259/491618557.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [ ]:
# 233 = positive
# 180 = negative
# 126 = neutral

In [122]:
df_1 = pd.read_csv("df_1_art.csv")
df_2 = pd.read_csv("df_2_art.csv")
df_3 = pd.read_csv("df_3_art.csv")

In [135]:
all_df = pd.concat([df_1, df_2, df_3])
# all_df.to_csv("articles_preprocessed.csv", index=False)

In [2]:
import pandas as pd
# all_df = pd.read_csv("articles_preprocessed.csv")

In [136]:
all_df['emotion'].value_counts()

emotion
Позитивная                                                             491
Отрицательная                                                          276
Нейтральная                                                            245
Негативная                                                              14
                                                                         8
Положительная                                                            1
Negativnaya                                                              1
Исходя из данного текста, эмоция может быть оценена как Позитивная.      1
Позитивная.                                                              1
Нейтральная.                                                             1
Ожидаемо, эмоция в данном тексте - нейтральная.                          1
Name: count, dtype: int64

In [110]:
prompt = "Тебе дан текст с сайта новостей. Нужно определить и выделить эмоцию у данного текст: Позитивная, Нейтральная, Негативная." \
         "\nТекст: Сотрудники УдмФИЦ УрО РАН принимают участие в III Конгрессе молодых ученых" \
         "Эмоция: Нейтральная" \
          f"\nТекст: {df_3['article'][290]}" \
         "\nЭмоция: "
df_3.loc[290, 'emotion'] = str(chat_gpt(prompt, "gpt-3.5-turbo"))


In [133]:
df_3

,link,title,article,кластер,emotion
0,https://www.d-kvadrat.ru/novosti/23576,Работу пансионата для престарелых «Теплый дом»...,Суд приостановил работу пансионата для престар...,3,Отрицательная
1,https://www.d-kvadrat.ru/novosti/25171,День памяти Героя: ЮРИЙ ГАГАРИН,"Ровно лет назад, марта года, в учебнотренир...",3,Отрицательная
2,https://www.d-kvadrat.ru/novosti/24818,21 кражу из магазина совершили два жителя Можги,Житель Можги набирал товары в одном из можгинс...,3,Отрицательная
3,https://www.d-kvadrat.ru/novosti/25782,Все 11 побратимов городов Удмуртии,В последнее воскресенье апреля отмечается Всем...,3,Нейтральная.
4,https://www.d-kvadrat.ru/novosti/25908,ИТОГИ НЕДЕЛИ,Итоги празднования в Ижевске мая и новости по...,3,Позитивная
...,...,...,...,...,...
343,https://www.d-kvadrat.ru/novosti/26396,Пара доллар-рубль перешла к консолидации в уз...,Пара долларрубль обновив в рамках недели макси...,4,Нейтральная
344,https://www.d-kvadrat.ru/novosti/25074,Индекс МосБиржи продолжил уверенно расти,"Индекс МосБиржи , продолжил уверенно расти, пр...",4,Нейтральная
345,https://www.d-kvadrat.ru/novosti/27527,Потенциал для восстановления котировок ОФЗ сох...,Потенциал для восстановления котировок ОФЗ сох...,4,Нейтральная
346,https://www.d-kvadrat.ru/novosti/27767,Российский рынок в пятницу продолжил восходящу...,Российский рынок в пятницу продолжил восходящу...,4,


In [95]:
df_3['emotion'].value_counts()

emotion
    348
Name: count, dtype: int64

In [115]:
df_3.to_csv("df_3_art.csv", index=False)

In [103]:
import time
from tqdm import tqdm

for i in tqdm(range(len(df_1))):
    try:
        prompt = "Тебе дан текст с сайта новостей. Нужно определить и выделить эмоцию у данного текст: Позитивная, Нейтральная, Отрицательная." \
         "\nТекст: Сотрудники УдмФИЦ УрО РАН принимают участие в III Конгрессе молодых ученых" \
         "Эмоция: Нейтральная" \
          f"\nТекст: {df_1['article'][i]}" \
         "\nЭмоция: "
        prompt_2 = "Тебе дан текст с сайта новостей. Нужно определить и выделить эмоцию у данного текст: Позитивная, Нейтральная, Отрицательная." \
         "\nТекст: Сотрудники УдмФИЦ УрО РАН принимают участие в III Конгрессе молодых ученых" \
         "Эмоция: Нейтральная" \
          f"\nТекст: {df_2['article'][i]}" \
         "\nЭмоция: "
        prompt_3 = "Тебе дан текст с сайта новостей. Нужно определить и выделить эмоцию у данного текст: Позитивная, Нейтральная, Отрицательная." \
         "\nТекст: Сотрудники УдмФИЦ УрО РАН принимают участие в III Конгрессе молодых ученых" \
         "Эмоция: Нейтральная" \
          f"\nТекст: {df_3['article'][i]}" \
         "\nЭмоция: "
        df_1.loc[i, 'emotion'] = str(chat_gpt(prompt, "gpt-3.5-turbo"))
        time.sleep(0.1)
        df_2.loc[i, 'emotion'] = str(chat_gpt(prompt_2, "gpt-3.5-turbo"))
        time.sleep(0.1)
        df_3.loc[i, 'emotion'] = str(chat_gpt(prompt_3, "gpt-3.5-turbo"))
    except:
        pass


100%|██████████| 346/346 [18:24<00:00,  3.19s/it]  


In [197]:
# Замени все Отрицательная на Негативная

all_df

,link,title,article,кластер,emotion
0,https://www.d-kvadrat.ru/novosti/24796,В Удмуртии простились с Константином Кутяновым...,марта в Удмуртии простились с Константином Ку...,0,Негативная
1,https://www.d-kvadrat.ru/novosti/26425,Спасатели оперативно освободили закрытого в ма...,"Вчера, июня, спасатели Удмуртии среагировали ...",0,Позитивная
2,https://www.d-kvadrat.ru/novosti/28679,Дети участников СВО и из «Движения первых» пос...,На выставке ребятам особенно запомнился павиль...,0,Позитивная
3,https://www.d-kvadrat.ru/novosti/27700,В Удмуртии начнут отмечать День мировой юстици...,Такое решение было принято сентября в ходе се...,0,Позитивная
4,https://www.d-kvadrat.ru/novosti/23134,Сегодня на сессии Госсовета Удмуртии депутаты ...,"На прошедшей сегодня, ноября, сессии Государс...",0,Позитивная
...,...,...,...,...,...
1035,https://www.d-kvadrat.ru/novosti/26396,Пара доллар-рубль перешла к консолидации в уз...,Пара долларрубль обновив в рамках недели макси...,4,Нейтральная
1036,https://www.d-kvadrat.ru/novosti/25074,Индекс МосБиржи продолжил уверенно расти,"Индекс МосБиржи , продолжил уверенно расти, пр...",4,Нейтральная
1037,https://www.d-kvadrat.ru/novosti/27527,Потенциал для восстановления котировок ОФЗ сох...,Потенциал для восстановления котировок ОФЗ сох...,4,Нейтральная
1038,https://www.d-kvadrat.ru/novosti/27767,Российский рынок в пятницу продолжил восходящу...,Российский рынок в пятницу продолжил восходящу...,4,Позитивная


In [185]:
prompt = "Тебе дан текст с сайта новостей. Нужно определить и выделить эмоцию у данного текст: Позитивная, Нейтральная, Негативная." \
         "\nТекст: Сотрудники УдмФИЦ УрО РАН принимают участие в III Конгрессе молодых ученых" \
         "Эмоция: Нейтральная" \
          f"\nТекст: {all_df['article'][573]}" \
         "\nЭмоция: "
all_df.loc[573, 'emotion'] = str(chat_gpt(prompt, "gpt-3.5-turbo"))


In [196]:
# уменшьи в три раза
all_df['emotion'].value_counts()

emotion
Позитивная     497
Негативная     293
Нейтральная    250
Name: count, dtype: int64

In [198]:
all_df.to_csv("sampling_arts_prep.csv", index=False)